# Install requirements
This code has been tested using Python 3.8 and TF 2.13 with GPU support. 

As of 9/1/2023 there are issues installing `tflite-model-maker` in Google Colab.

In [ ]:
!pip install --upgrade pip
!pip install tensorflow==2.13.*

In [ ]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

In [ ]:
!pip install pycocotools

# Import required libraries

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

# Train the OD model

## 1. Preprocess and load the image dataset
The images used to train the model are PNG, by default, with RGBA channels. However, DataLoader is expecting a RGB JPEG, so we convert the files to RGB.

In [ ]:
TRAIN_IMG_PATH = os.path.join('workspace','images','train')
TEST_IMG_PATH = os.path.join('workspace','images','test')

In [ ]:
import glob
from PIL import Image
files = glob.glob("{}/*".format(TRAIN_IMG_PATH))
for file in files:
    if ".jpeg" in file:
        image = Image.open(file)
        if image.format not in ["JPG","JPEG"]:
            print(file)
            image.convert('RGB').save(file,"JPEG")
            
files = glob.glob("{}/*".format(TEST_IMG_PATH))
for file in files:
    if ".jpeg" in file:
        image = Image.open(file)
        if image.format not in ["JPG","JPEG"]:
            print(file)
            image.convert('RGB').save(file,"JPEG")

We create the TF Records for the training and testing datasets using our label map

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    TRAIN_IMG_PATH,
    TRAIN_IMG_PATH,
    ['ACCOUNT','PASSWORD']
)

In [ ]:
test_data = object_detector.DataLoader.from_pascal_voc(
    TEST_IMG_PATH,
    TEST_IMG_PATH,
    ['ACCOUNT','PASSWORD']
)

## 3. Select model architecture

In [ ]:
spec = model_spec.get('efficientdet_lite0')

## Train the model

In [ ]:
model = object_detector.create(
    train_data,
    model_spec = spec,
    batch_size = 4,
    train_whole_model = True,
    epochs = 100,
    validation_data = test_data
)

In [ ]:
model.evaluate(test_data)